In [1]:
import syft as sy
from syft.core.node.new.api import NodeView

In [2]:
worker = sy.Worker.named("test-domain-1", processes=1)
domain_client = worker.root_client

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
results = domain_client.api.services.dataset.get_all()
results

,type,id,name,url
0,syft.core.node.new.dataset.Dataset,ecb440770edd422f8fab0e6a5338010d,Canada Trade Value,https://github.com/OpenMined/datasets/tree/mai...


In [4]:
assert len(results) == 1

In [5]:
dataset = results[-1]

In [6]:
dataset

```python
Syft Dataset: Canada Trade Value
Assets:
	canada_trade_flow: all the datas
Citation: Person, place or thing
URL: https://github.com/OpenMined/datasets/tree/main/trade_flow
Description: Canada Trade Data

```

In [7]:
mock = dataset.assets[0].mock
mock

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
10,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Bangladesh,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,227222,0
11,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Haiti,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,14748,0
12,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Guatemala,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1314,0
13,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Iraq,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1825,0
14,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Israel,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1063627,0
15,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Italy,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,8359327,0
16,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Jordan,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,16858,0
17,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Rep. of Moldova,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,29897,0
18,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Nigeria,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,22235,0
19,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Slovenia,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1328,0


In [42]:
dataset.assets[0].data

'Permission: <READ: 7a772a44717b4c4bb149cdfd377689d8 as 8ba9bf578676d73d35631aef51481e7892bb95ebbb6c1f08ec6268b4e60328ee> denied'

In [8]:
mock["Trade Value (US$)"].sum()

9738381

In [9]:
@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))

In [10]:
result = sum_trade_value_mil(trade_data=mock)
result

(9.738381, -7.458751914827758)

In [11]:
assert result[0] == 9.738381

In [12]:
assert isinstance(result[1], float)

In [13]:
assert len(sum_trade_value_mil.kwargs) == 1

In [14]:
node_view = NodeView(node_name =domain_client.api.node_name,verify_key =domain_client.api.signing_key.verify_key)
assert node_view in sum_trade_value_mil.kwargs

In [15]:
assert "trade_data" in sum_trade_value_mil.kwargs[node_view]

In [16]:
assert len(sum_trade_value_mil.outputs) == 0

In [17]:
assert sum_trade_value_mil.input_policy.inputs[node_view]["trade_data"] == mock.id

In [18]:
sum_trade_value_mil.code

'@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),\n                  output_policy=sy.SingleExecutionExactOutput())\ndef sum_trade_value_mil(trade_data):\n    import pandas as pd\n    from opendp.mod import enable_features\n    enable_features(\'contrib\')\n    from opendp.measurements import make_base_laplace\n    aggregate = 0.\n    base_lap = make_base_laplace(scale=5.)\n    noise = base_lap(aggregate)\n\n    df = trade_data\n    total = df["Trade Value (US$)"].sum()\n    return (float(total / 1_000_000), float(noise))\n'

Get a `guest client (data scientist)` who will submit some code to be executed

In [19]:
guest_domain_client = domain_client.guest()

In [20]:
guest_credentials = guest_domain_client.credentials
guest_credentials

<SigningKey for: 34bd9af17d00a1bed6a426f4c42b47f7c387b2d97f050fb8ee28cd8b17200c37>

In [21]:
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")

<class 'syft.core.node.new.response.SyftSuccess'>: jane@caltech.edu User successfully registered !!!

In [22]:
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: PythonConnection>

In [25]:
assert guest_domain_client.credentials != guest_credentials

True

In [24]:
guest_domain_client.api.services.code.request_code_execution(sum_trade_value_mil)

```python
class Request:
  id: str = 692dde01db084cf4af4eb6cfde6b70be
  requesting_user_verify_key: str = 8ba9bf578676d73d35631aef51481e7892bb95ebbb6c1f08ec6268b4e60328ee
  approving_user_verify_key: str = None
  request_time: str = 2023-03-14 01:45:23
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "90ef93fa64c6cb70482b598c2fd75306b9ff3c0ce67f7380c77c90d031e10214"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange]

```

In [26]:
guest_domain_client.code

,type,id,status,service_func_name
0,syft.core.node.new.user_code.UserCode,161cf7c383024becbcdcb0e5e7a225fc,"{NodeView(node_name='test-domain-1', verify_ke...",sum_trade_value_mil


In [27]:
assert len(guest_domain_client.api.services.code.get_all()) == 1

In [28]:
new_project = sy.Project(name="My Cool UN Project")

In [29]:
proj_desc = """Hi, I want to calculate the trade volume in million's with my cool code."""
new_project.set_description(proj_desc)

In [30]:
submitted_code = guest_domain_client.code[0]

In [31]:
submitted_code

```python
class UserCode:
  id: str = 161cf7c383024becbcdcb0e5e7a225fc
  user_verify_key: str = 8ba9bf578676d73d35631aef51481e7892bb95ebbb6c1f08ec6268b4e60328ee
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))
"
  input_policy: str = id=<UID: afae9d5ed99c402d863d7c201e3eb341> inputs={NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'trade_data': <UID: 7a772a44717b4c4bb149cdfd377689d8>}} node_uid=<UID: 7bca415d13ed1ec841f0d0aede098dbb>
  output_policy: str = id=<UID: 2800e6e8a01c45caa9cafbd0ec9c1de9> outputs=[] state_type=<class 'syft.core.node.new.user_code.OutputPolicyStateExecuteOnce'>
  output_policy_state: str = id=<UID: 0ca52d22a1904f2c912750b03606fa11> output_history=[] count=0 limit=1
  parsed_code: str = "

def user_func_sum_trade_value_mil_8ba9bf578676d73d35631aef51481e7892bb95ebbb6c1f08ec6268b4e60328ee_5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0(trade_data):

    def sum_trade_value_mil(trade_data):
        import pandas as pd
        from opendp.mod import enable_features
        enable_features('contrib')
        from opendp.measurements import make_base_laplace
        aggregate = 0.0
        base_lap = make_base_laplace(scale=5.0)
        noise = base_lap(aggregate)
        df = trade_data
        total = df['Trade Value (US$)'].sum()
        return (float((total / 1000000)), float(noise))
    result = sum_trade_value_mil(trade_data=trade_data)
    return result
"
  service_func_name: str = "sum_trade_value_mil"
  unique_func_name: str = "user_func_sum_trade_value_mil_8ba9bf578676d73d35631aef51481e7892bb95ebbb6c1f08ec6268b4e60328ee_5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0"
  user_unique_func_name: str = "user_func_sum_trade_value_mil_8ba9bf578676d73d35631aef51481e7892bb95ebbb6c1f08ec6268b4e60328ee"
  code_hash: str = "5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0"
  signature: str = (trade_data) -> Dict[str, Any]
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  enclave_metadata: str = None

```

In [32]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [33]:
new_project.changes

,type,id
0,syft.core.node.new.project.ObjectPermissionChange,c9dd53a3dca4454d875ba628e28b810f


In [34]:
new_project

```python
class ProjectSubmit:
  id: str = None
  name: str = "My Cool UN Project"
  description: str = "Hi, I want to calculate the trade volume in million's with my cool code."
  changes: str = [syft.core.node.new.project.ObjectPermissionChange]

```

In [35]:
guest_domain_client.submit_project(new_project)

<class 'syft.core.node.new.response.SyftSuccess'>: Project Submitted

In [36]:
guest_domain_client._api = None
_ = guest_domain_client.api

In [37]:
result = guest_domain_client.api.services.code.sum_trade_value_mil(trade_data=mock.id)
result

<class 'syft.core.node.new.response.SyftNotReady'>: <class 'syft.core.node.new.user_code.UserCode'> Your code is waiting for approval: {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}

In [38]:
assert isinstance(result, sy.SyftNotReady)